# Bedrock AgentCore Code Interpreter tools with LangGraph React Agent

This notebook demonstrates how to use the [Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/) based code interpreter toolkit with a LangGraph React Agent to perform code execution tasks. This toolkit provides a set of tools for running code, executing shell commands, and managing files in a secure environment.

## Setup and Installation

First, make sure you have the required packages installed:

In [ ]:
%pip install -q langgraph langchain 'langchain-aws[tools]'

## Import Required Libraries

In [5]:
import logging

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_aws.tools import create_code_interpreter_toolkit

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Create Code Interpreter Toolkit

In [6]:
# Create the code interpreter toolkit
# This is an async function as it sets up the tools
toolkit, code_tools = await create_code_interpreter_toolkit(region="us-west-2")

# Display available tools
print(f"Available code interpreter tools: {[tool.name for tool in code_tools]}")

Available code interpreter tools: ['execute_code', 'execute_command', 'read_files', 'list_files', 'delete_files', 'write_files', 'upload_file', 'install_packages', 'start_command_execution', 'get_task', 'stop_task']


## Initialize LLM

Set up the language model that will power our agent. We'll use Claude 3.5 Haiku through Bedrock.

In [2]:
# Initialize the language model using bedrock_converse provider
provider = "bedrock_converse"
model_id = "us.anthropic.claude-opus-4-5-20251101-v1:0"
model_with_provider = f"{provider}:{model_id}"

# Create the model instance
model = init_chat_model(model_with_provider)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


## Create React Agent

Now we'll create a React agent using LangGraph's prebuilt agent. The React agent uses a reasoning and acting approach to solve tasks step by step.

In [3]:
# Create the React agent with code interpreter tools
agent = create_agent(
    model,
    tools=code_tools,
    # Customize the agent prompt for code execution tasks
    system_prompt="""You are a code execution assistant that can run Python code, execute shell commands, and manage files.
    Use the available code interpreter tools to complete programming tasks.
    
    Available tools:
    - execute_code: Run code in the environment (primarily Python)
    - execute_command: Run shell commands
    - read_files: Read content of files
    - list_files: List files in directories
    - delete_files: Remove files
    - write_files: Create or update files
    - upload_file: Upload files with semantic descriptions
    - install_packages: Install Python packages
    - start_command_execution: Start long-running commands asynchronously
    - get_task: Check status of async tasks
    - stop_task: Stop running tasks
    
    Follow these steps for each task:
    1. Understand the problem requirements
    2. Write code or commands to solve the problem
    3. Execute the code and analyze results
    4. Refine your approach based on results if needed
    5. Provide a clear explanation of the solution"""
)


## Execute the Agent

Now let's run the agent on a code execution task. We'll set up a function to execute tasks and print the results.

In [4]:
async def run_code_interpreter_agent(query: str, session_id: str = "code_session1"):
    """
    Run the code interpreter agent on a specific query with session tracking
    
    Args:
        query: The task to perform
        session_id: Unique identifier for the code interpreter session
        
    Returns:
        Agent response
    """
    try:
        # Configure the session ID for thread-aware tools
        config = {
            "configurable": {
                "thread_id": session_id
            }
        }
        
        # Invoke the agent with the query
        result = await agent.ainvoke(
            {
                "messages": [{
                    "role": "human",
                    "content": query
                }]
            },
            config=config
        )
        
        return result
    except Exception as e:
        logger.error(f"Error running code interpreter agent: {e}")
        raise

## Example Tasks

Let's run some example code execution tasks to demonstrate the agent's capabilities.

In [10]:
# Example 1: Execute basic Python code
task1 = "Create a Python function that calculates the factorial of a number, then test it with n=5"

result1 = await run_code_interpreter_agent(task1)
print("Task 1 Result:")
print(result1["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:langchain_aws.tools.local_code_interpreter_client:Starting code interpreter session...
INFO:langchain_aws.tools.local_code_interpreter_client:✅ Session started: 01KE0743SD51BW00GCVDENESST
INFO:langchain_aws.tools.code_interpreter_toolkit:Started code interpreter with session_id:01KE0743SD51BW00GCVDENESST for thread:code_session1
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 1 Result:
I've created a `factorial` function and tested it. Here's a summary:

## The Factorial Function

The function calculates n! (n factorial) using an iterative approach:

```python
def factorial(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    if n == 0 or n == 1:
        return 1
    else:
        result = 1
        for i in range(2, n + 1):
            result *= i
        return result
```

## Key Features:
- **Input validation**: Raises an error for negative numbers
- **Base cases**: Returns 1 for both 0! and 1!
- **Iterative calculation**: Multiplies all integers from 2 to n

## Test Result:
- **factorial(5) = 120** ✓

This is correct because: 5! = 5 × 4 × 3 × 2 × 1 = 120

The additional tests show the function works correctly for values 0 through 7.


In [11]:
# Example 2: Create and manipulate files
task2 = """Create a CSV file named 'data.csv' with the following data:
Name,Age,City
John,30,New York
Alice,25,Boston
Bob,35,Chicago
Emma,28,Seattle

Then write a Python script to read this CSV file and calculate the average age."""

result2 = await run_code_interpreter_agent(task2)
print("Task 2 Result:")
print(result2["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 2 Result:
I've completed both tasks:

1. **Created `data.csv`** with the following content:
   ```
   Name,Age,City
   John,30,New York
   Alice,25,Boston
   Bob,35,Chicago
   Emma,28,Seattle
   ```

2. **Python script** that reads the CSV and calculates the average age:
   - Used Python's built-in `csv` module with `DictReader` to read the file
   - Extracted all ages: [30, 25, 35, 28]
   - Calculated the sum: 118
   - Divided by the number of people: 4
   - **Average age: 29.50 years**


In [ ]:
# Example 3: Upload file with semantic description
task_upload = """Use the upload_file tool to create a JSON configuration file called 'config.json' with the following content:
{
    "app_name": "DataAnalyzer",
    "version": "1.0",
    "features": ["csv_import", "visualization", "export"]
}

Include a description that explains what this config file is for."""

result_upload = await run_code_interpreter_agent(task_upload)
print("Upload File Result:")
print(result_upload["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.tools.local_code_interpreter_client:Uploading file: config.json (Configuration file for the DataAnalyzer application. Contains app metadata (name, version) and a list of enabled features including CSV import, data visualization, and export capabilities.)
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Upload File Result:
I've created the `config.json` file with the specified content. The file contains:

- **app_name**: "DataAnalyzer" - the name of the application
- **version**: "1.0" - the current version
- **features**: An array of three enabled features:
  - `csv_import` - for importing CSV files
  - `visualization` - for data visualization capabilities
  - `export` - for exporting data

The semantic description I included explains that this is a configuration file for the DataAnalyzer application, containing metadata and feature flags. This description helps document what the file is for and makes it easier to understand its purpose in the project.


In [ ]:
# Example 4: Check what packages are available
task_packages = """Use execute_command to run 'pip list' and show me what packages 
are already installed in the environment."""

result_packages = await run_code_interpreter_agent(task_packages)
print("Install Packages Result:")
print(result_packages["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Install Packages Result:
Here's a summary of the packages already installed in the environment:

## Pre-installed Packages

The environment comes with a comprehensive set of packages across various categories:

### Data Science & Machine Learning
- **pandas** (2.3.1), **numpy** (1.26.4), **scipy** (1.16.0)
- **scikit-learn** (1.5.0), **xgboost** (2.0.3)
- **torch** (2.3.0), **torchvision** (0.18.0), **torchaudio** (2.3.0)
- **statsmodels** (0.14.5)

### Visualization
- **matplotlib** (3.9.0), **plotly** (5.22.0), **seaborn** (not listed but usually included)
- **bokeh** (2.4.3)

### Data Processing
- **pyarrow** (17.0.0), **openpyxl** (3.1.3), **xlrd** (2.0.1)
- **beautifulsoup4** (4.12.3), **lxml** (5.2.2)
- **duckdb** (1.3.2)

### Image/Video Processing
- **opencv-python** (4.10.0.82), **pillow** (10.3.0)
- **imageio** (2.34.1), **moviepy** (1.0.3), **scikit-image** (0.23.2)

### PDF & Document Processing
- **pypdf** (6.2.0), **PyPDF2** (3.0.1), **pdfplumber** (0.11.0)
- **python-doc

In [ ]:
# Example 5: Generate plots
task5 = """Generate a matplotlib plot showing a sine wave over the range [0, 2π] and save it as 'sine_wave.png'. 
Then create another plot showing both sine and cosine waves on the same graph and save it as 'trig_functions.png'."""

result3 = await run_code_interpreter_agent(task3)
print("Task 3 Result:")
print(result3["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:bedrock_agentcore.tools.code_interpreter_client:Starting code interpreter session...
INFO:bedrock_agentcore.tools.code_interpreter_client:✅ Session started: 01KE2YWEMK410XEPYZQWKW9X9A
INFO:langchain_aws.tools.code_interpreter_toolkit:Started code interpreter with session_id:01KE2YWEMK410XEPYZQWKW9X9A for thread:code_session1
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 3 Result:
I've created both plots as requested:

1. **`sine_wave.png`** - Shows a single sine wave over the range [0, 2π]:
   - Blue line representing sin(x)
   - X-axis labeled with radians (0, π/2, π, 3π/2, 2π)
   - Grid lines for easier reading
   - Proper title and labels

2. **`trig_functions.png`** - Shows both sine and cosine waves on the same graph:
   - Blue line for sin(x)
   - Red line for cos(x)
   - Legend to distinguish between the two functions
   - Same x-axis range [0, 2π] with radian labels
   - Grid lines and proper formatting

Both plots have been saved at 150 DPI resolution with a figure size of 10x6 inches, making them suitable for presentations or documentation.


In [ ]:
task6 = "Show me all the files present in the code interpreter environment"

result4 = await run_code_interpreter_agent(task4)
print("Task 4 Result:")
print(result4["messages"][-1].content)

INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response
INFO:langchain_aws.chat_models.bedrock_converse:Using Bedrock Converse API to generate response


Task 4 Result:
Great! Let me break down the files in the current directory:

1. Directories:
   - `.cache`
   - `.config`
   - `.ipython`
   - `log`

2. Files:
   - `data.csv`
   - `sine_wave.png`
   - `trig_functions.png`

There are a few interesting things to note:
- There's a CSV file called `data.csv`
- Two image files: `sine_wave.png` and `trig_functions.png`
- Some hidden configuration directories

Would you like me to show you the contents of any of these files or provide more details about them?


## Clean Up Resources

Always clean up code interpreter resources when done to avoid unnecessary resource usage.

In [ ]:
# Clean up code interpreter resources
await toolkit.cleanup()
print("Code interpreter resources cleaned up successfully")

INFO:langchain_aws.tools.code_interpreter_toolkit:All code interpreter sessions cleaned up


Code interpreter resources cleaned up successfully
